In [1]:
import pandas as pd

In [5]:
nRowsRead = 1000 # specify 'None' if want to read whole file
# training.1600000.processed.noemoticon.csv may have more rows in reality, but we are only loading/previewing the first 1000 rows
df1 = pd.read_csv('data.csv', delimiter=',', nrows = nRowsRead)
df1.dataframeName = 'data.csv'
df1.columns = ['target', 'ids', 'date', 'flag', 'user', 'text']
# print(f'There are {nRow} rows and {nCol} columns')

There are 1000 rows and 6 columns


In [15]:
df1.shape
df1.isnull().sum()
df1.head(500)


,target,ids,date,flag,user,text
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew
...,...,...,...,...,...,...
495,0,1467935121,Mon Apr 06 22:53:14 PDT 2009,NO_QUERY,SicklyInfected,I wanna be in a punk rock band again
496,0,1467935189,Mon Apr 06 22:53:15 PDT 2009,NO_QUERY,AshleighF,@jyesmith That's a lot of angst for a Tuesday ...
497,0,1467935271,Mon Apr 06 22:53:19 PDT 2009,NO_QUERY,andreakuefler,I'm very glad Britney isn't crazy anymore! Tha...
498,0,1467935345,Mon Apr 06 22:53:18 PDT 2009,NO_QUERY,mmmelissa,has a huuuge headache! omg I feel like crap!


In [18]:
#Libraries of NLTK
import re,string
import tensorflow as tf

import nltk
from nltk.corpus import stopwords
from nltk.corpus import wordnet
from nltk.stem import PorterStemmer,WordNetLemmatizer

#cleaning the data from unnecessary data

#replacing url with string URL
def replace_url(text):
    return re.sub('https?:\/\/\S*|www\.\S+','URL',text)

#removing html
def remove_html(text):
    return re.sub('<.*?>','',text)

#replacing mentions with string user
def replace_mentions(text):
    return re.sub('@\S*','user',text,flags=re.IGNORECASE)

#replacing numbers with string number
def replace_num(text):
    return re.sub('^[+-]*?\d{1,3}[- ]*?\d{1,10}|\d{10}','NUMBER',text)

#replacing <3 with sring heart
def replace_heart(text):
    return re.sub('<3','HEART', text)

#removing alphanumeric characters eg-XYZ123ABC
def remove_alphanumeric(text):
    return re.sub('\w*\d+\w*','',text)

#removing all english stopwords 
def remove_stopwords(text):
    text = ' '.join([word for word in text.split() if word not in stopwords.words("english")])
    return text

#removing punctuations
def remove_punctuations(text):
    text=''.join([word for word in text if word not in string.punctuation])
    return text

#reducing words to thier root form
def lemmatization(text):
    lm= WordNetLemmatizer()
    text = ' '.join([lm.lemmatize(word, pos='v') for word in text.split()])
    return text

In [19]:
def clean_text(text):
    text=str(text).lower()
    text = replace_url(text)
    text = remove_html(text)
    text = replace_mentions(text)
    text = replace_num(text)
    text = replace_heart(text)
    text = remove_alphanumeric(text)
    text = remove_stopwords(text)
    text=remove_punctuations(text)
    #text=stemming(text)
    text=lemmatization(text)
    return text

In [20]:
df1['text1']=df1['text'].apply(clean_text)

LookupError: 
**********************************************************************
  Resource [93mstopwords[0m not found.
  Please use the NLTK Downloader to obtain the resource:

  [31m>>> import nltk
  >>> nltk.download('stopwords')
  [0m
  For more information see: https://www.nltk.org/data.html

  Attempted to load [93mcorpora/stopwords[0m

  Searched in:
    - 'C:\\Users\\hamza.qureshi/nltk_data'
    - 'c:\\Users\\hamza.qureshi\\AppData\\Local\\anaconda3\\envs\\generalEnv\\nltk_data'
    - 'c:\\Users\\hamza.qureshi\\AppData\\Local\\anaconda3\\envs\\generalEnv\\share\\nltk_data'
    - 'c:\\Users\\hamza.qureshi\\AppData\\Local\\anaconda3\\envs\\generalEnv\\lib\\nltk_data'
    - 'C:\\Users\\hamza.qureshi\\AppData\\Roaming\\nltk_data'
    - 'C:\\nltk_data'
    - 'D:\\nltk_data'
    - 'E:\\nltk_data'
**********************************************************************
